<a href="https://colab.research.google.com/github/mostafa-ja/graph-tutorial/blob/master/Illegal_Bitcoin_Transactions(GCN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Install Kaggle API.
!pip install kaggle
# Run the following code to configure the path to “kaggle.json”
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle datasets download -d ellipticco/elliptic-data-set
!unzip elliptic-data-set.zip

elliptic-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  elliptic-data-set.zip
replace elliptic_bitcoin_dataset/elliptic_txs_classes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [48]:
!pip install torch_geometric

In [49]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'axes.facecolor':'dimgrey', 'grid.color':'lightgrey'})

import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn.functional as F
import torch.nn as nn
#import torch_scatter
from torch_geometric.data import Data
print(torch.__version__)

# # The PyG built-in GCNConv
# from torch_geometric.nn import GCNConv
from torch_geometric.nn.conv import MessagePassing
import torch_geometric.transforms as T
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax, degree

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
import scipy.sparse as scsp
from sklearn.cluster import KMeans
import copy

2.0.1+cu118


In [50]:
path = '/content'

edges = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
classes = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
features = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

In [51]:
features.head(2)

,0,1,2,3,4,5,6,7,8,9
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162097
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162112


In [52]:
classes.head(2)

,txId,class
0,230425980,unknown
1,5530458,unknown


In [53]:
edges.head(2)

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [54]:
# rename features
rename_dict = dict(
    zip(
        range(0, 167),
        ["txId", "time_step"]
        + [i for i in range(2, 168)]
    )
)
features.rename(columns=rename_dict, inplace=True)

In [55]:
features.head()

,txId,time_step,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [56]:
# rename the classes to ints that can be handled by pytorch as labels
classes["class"] = classes["class"].replace(
    {"unknown": -1,  # unlabeled nodes
     "2": 0,  # labeled licit nodes
     #"1": 1,  # labeled illicit nodes
    }
).astype(int)

In [57]:
# Merge features and labels
df = features.merge(classes, how="left", left_on="txId", right_on="txId")
df.head()

,txId,time_step,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,class
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,-1
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,-1
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,-1
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,0
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,-1


In [58]:
class_column = df.pop('class')


In [59]:
df.insert(1, 'class', class_column)

In [60]:
df.head()

,txId,class,time_step,2,3,4,5,6,7,8,...,157,158,159,160,161,162,163,164,165,166
0,230425980,-1,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,-1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,-1,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,-1,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [61]:
# Sort the DataFrame and reset index
#df.sort_values("txId", inplace=True)
#df.reset_index(drop=True, inplace=True)

# Check the length of df and classes
if len(df) != len(classes):
    print("Error: Lengths of df and classes do not match.")

# Drop unclassified and isolated nodes
classified_nodes = set(classes[classes["class"] != -1]["txId"].values)
classified_edges = edges[(edges["txId1"].isin(classified_nodes)) & (edges["txId2"].isin(classified_nodes))].copy()
non_isolated_nodes = set(classified_edges["txId1"].values).union(classified_edges["txId2"].values)
classified_df = df[df["txId"].isin(non_isolated_nodes)].copy()


In [65]:
classified_df.reset_index(drop=True, inplace=True)
classified_df.head()

,txId,class,time_step,2,3,4,5,6,7,8,...,157,158,159,160,161,162,163,164,165,166
0,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
1,232029206,0,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,...,-0.577099,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792
2,232344069,0,1,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
3,27553029,0,1,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.539735,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
4,3881097,0,1,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,...,-0.577099,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984


In [67]:
classified_df.info

<bound method DataFrame.info of             txId  class  time_step         2         3         4          5  \
0      232438397      0          1  0.163054  1.963790 -0.646376  12.409294   
1      232029206      0          1 -0.005027  0.578941 -0.091383   4.380281   
2      232344069      0          1 -0.147852 -0.184668 -1.201369  -0.121970   
3       27553029      0          1 -0.151357 -0.184668 -1.201369  -0.121970   
4        3881097      0          1 -0.172306 -0.184668 -1.201369   0.028105   
...          ...    ...        ...       ...       ...       ...        ...   
35869   80329479      0         49 -0.159293 -0.037276  1.018602  -0.121970   
35870  158406298      0         49 -0.172962 -0.126566  1.018602  -0.121970   
35871  158375075      1         49 -0.170412 -0.078164  1.018602   0.028105   
35872  147478192      0         49 -0.093732 -0.116160  1.018602  -0.121970   
35873  158375402      1         49 -0.172014 -0.078182  1.018602   0.028105   

              6    

In [66]:
classified_edges.reset_index(drop=True, inplace=True)
classified_edges.head()

,txId1,txId2
0,232344069,27553029
1,3881097,232457116
2,232051089,232470704
3,230473487,7089694
4,231182296,14660781


In [68]:
classified_edges.info

<bound method DataFrame.info of            txId1      txId2
0      232344069   27553029
1        3881097  232457116
2      232051089  232470704
3      230473487    7089694
4      231182296   14660781
...          ...        ...
36619  194020062   47521535
36620  158574502  109383451
36621  158594124  157631640
36622  157631640   21644119
36623  158365409  157930723

[36624 rows x 2 columns]>

In [69]:
features =  torch.FloatTensor(classified_df.iloc[:, 2:].to_numpy())
labels = classified_df.iloc[:, 1].to_numpy()
labels = torch.LongTensor(labels.reshape(len(labels),))
nodes_id = classified_df.iloc[:, 0].to_numpy()

In [71]:
features.shape

torch.Size([35874, 166])

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_size =
test_size =

In [ ]:
# Create a dictionary that maps nodeId to index in the dataframe.
id2idx = {}
for i in range(classified_df.shape[0]):
    id2idx[classified_df.iloc[i, 0]] = i

# Construct edge_index with same node indexing as in features and labels
edge_idx = np.zeros((2, new_edges.shape[0]), dtype = np.int64)
for index in range(new_edges.shape[0]):
    node1 = id2idx[new_edges.iloc[index, 0]]
    node2 = id2idx[new_edges.iloc[index, 1]]
    edge_idx[:, index] = [node1, node2]
edge_index = torch.LongTensor(edge_idx)

train_index, test_index = train_test_split(np.arange(labels.shape[0]),
                                            test_size = 1 - train_size,
                                            random_state = 42)
val_index, test_index = train_test_split(test_index,
                                          test_size = test_size / (1 - train_size),
                                          random_state = 42)

# Construct the training, validation, test set and store
# in a dictionary, 'data'.
data = Data(x = features, edge_index = edge_index, y = labels).to(device)
